<a href="https://colab.research.google.com/github/DanaKailani/MonkeyBreedClassifier/blob/main/MonkeyBreedCLF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.chdir('/content/drive/My Drive')
!pwd

/content/drive/My Drive


In [ ]:
from keras.applications import MobileNet
from keras.models import Sequential,Model 
from keras.layers import Dense,Dropout,Activation,Flatten,GlobalAveragePooling2D
from keras.layers import Conv2D,MaxPooling2D,ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
# MobileNet is designed to work with images of dim 224,224
img_rows,img_cols = 224,224

In [ ]:
MobileNet = MobileNet(weights='imagenet',include_top=False,input_shape=(img_rows,img_cols,3))
 # we will use only the main cnn architecture
 # include top=false because we want to remove the last layer(dense layer) we will define it by our own
 # By specifying the include_top=False argument, you load a network that doesn't include the
 # classification layers at the top, which is ideal for feature extraction.

 #MobileNet is already trained on thousands of classes so we don't want to train the weights again on our dataset

In [ ]:
# Here we freeze the last 4 layers
# Layers are set to trainable as True by default

for layer in MobileNet.layers:
    layer.trainable = False #we want to leave those untrained and train the layers we define by our own

# we print our layers to visualize and check which layers are not trainable
for (i,layer) in enumerate(MobileNet.layers):
    print(str(i),layer.__class__.__name__,layer.trainable)

0 InputLayer False
1 ZeroPadding2D False
2 Conv2D False
3 BatchNormalization False
4 ReLU False
5 DepthwiseConv2D False
6 BatchNormalization False
7 ReLU False
8 Conv2D False
9 BatchNormalization False
10 ReLU False
11 ZeroPadding2D False
12 DepthwiseConv2D False
13 BatchNormalization False
14 ReLU False
15 Conv2D False
16 BatchNormalization False
17 ReLU False
18 DepthwiseConv2D False
19 BatchNormalization False
20 ReLU False
21 Conv2D False
22 BatchNormalization False
23 ReLU False
24 ZeroPadding2D False
25 DepthwiseConv2D False
26 BatchNormalization False
27 ReLU False
28 Conv2D False
29 BatchNormalization False
30 ReLU False
31 DepthwiseConv2D False
32 BatchNormalization False
33 ReLU False
34 Conv2D False
35 BatchNormalization False
36 ReLU False
37 ZeroPadding2D False
38 DepthwiseConv2D False
39 BatchNormalization False
40 ReLU False
41 Conv2D False
42 BatchNormalization False
43 ReLU False
44 DepthwiseConv2D False
45 BatchNormalization False
46 ReLU False
47 Conv2D False
48 Batc

In [ ]:
def addTopModelMobileNet(bottom_model, num_classes):
    """creates the top or head of the model that will be 
    placed ontop of the bottom layers"""

    top_model = bottom_model.output #we will take the mobilenet output and feed it as an input to the fully connected classifer(top model)
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    
    top_model = Dense(1024,activation='relu')(top_model)
    
    top_model = Dense(512,activation='relu')(top_model) #relu for binary classification
    
    top_model = Dense(num_classes,activation='softmax')(top_model)
    #The final dense layers will be the number of classes
    # we used softmax because it is categorical classification

    return top_model

In [ ]:
num_classes = 10 

FC_Head = addTopModelMobileNet(MobileNet, num_classes)

model = Model(inputs = MobileNet.input, outputs = FC_Head) #Model is same as Sequential

print(model.summary())
#Weights and parameters will only be updated in the last layers we defined

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128 

In [ ]:
train_data_dir = '/content/drive/My Drive/Monkey Breed/training/training'
validation_data_dir = '/content/drive/My Drive/Monkey Breed/validation/validation'

In [ ]:
#we are going to use data augmentation technique which expands our dataset by
#taking the data and shifting/rotating.. it
#to achieve higher accuracy (used for small data)

train_datagen = ImageDataGenerator(
                    rescale=1./255, #normalization
                    rotation_range=30, #r30 degrees to left and 30 degrees to right
                    width_shift_range=0.3,
                    height_shift_range=0.3,
                    horizontal_flip=True,
                    fill_mode='nearest') #if we are shifting to left the right area will lose some pixels
                                         #so we will fill it out

validation_datagen = ImageDataGenerator(rescale=1./255) #in validation we don't need huge amount of data so we are only normalizing the images

batch_size = 32

In [ ]:
train_generator = train_datagen.flow_from_directory(
                        train_data_dir,
                        target_size = (img_rows,img_cols), #224x244
                        batch_size = batch_size,
                        class_mode = 'categorical') 

validation_generator = validation_datagen.flow_from_directory(
                            validation_data_dir,
                            target_size=(img_rows,img_cols),
                            batch_size=batch_size,
                            class_mode='categorical')

Found 1098 images belonging to 10 classes.
Found 272 images belonging to 10 classes.


In [ ]:
from keras.optimizers import RMSprop,Adam 
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau


checkpoint = ModelCheckpoint(
                             'monkey_breed_mobilNet2.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

earlystop = EarlyStopping(
                          monitor='val_loss',
                          min_delta=0,
                          patience=10,
                          verbose=1,restore_best_weights=True)

learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                            patience=5, 
                                            verbose=1, 
                                            factor=0.2, 
                                            min_lr=0.0001)

callbacks = [earlystop,checkpoint,learning_rate_reduction]

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy']
              )

nb_train_samples = 1098
nb_validation_samples = 272  

epochs = 25
batch_size = 32

In [ ]:
history = model.fit_generator(
            train_generator,
            steps_per_epoch=nb_train_samples//batch_size,
            epochs=epochs,
            callbacks=callbacks,
            validation_data=validation_generator,
            validation_steps=nb_validation_samples//batch_size)

#Output:
# at some point validation accuracy may be higher than training accuracy but this is not overfitting
#because during training there is a dropout session (dropout 50% of the layer throughout the training) so it suffers from this dropout
#so it suffers also from lower accuracy.
#but during validation there is no dropout layer, our neural net is open to classify objects.
#that's why at some point our validation acc is high and training acc is a little bit less

Epoch 1/25
34/34 [==============================] - 236s 7s/step - loss: 1.2717 - accuracy: 0.5600 - val_loss: 0.6807 - val_accuracy: 0.7969

Epoch 00001: val_loss improved from inf to 0.68069, saving model to monkey_breed_mobilNet2.h5
Epoch 2/25
34/34 [==============================] - 45s 1s/step - loss: 0.4541 - accuracy: 0.8508 - val_loss: 0.2478 - val_accuracy: 0.8792

Epoch 00002: val_loss improved from 0.68069 to 0.24776, saving model to monkey_breed_mobilNet2.h5
Epoch 3/25
34/34 [==============================] - 39s 1s/step - loss: 0.3393 - accuracy: 0.8799 - val_loss: 0.0971 - val_accuracy: 0.9250

Epoch 00003: val_loss improved from 0.24776 to 0.09708, saving model to monkey_breed_mobilNet2.h5
Epoch 4/25
34/34 [==============================] - 39s 1s/step - loss: 0.2263 - accuracy: 0.9240 - val_loss: 0.3854 - val_accuracy: 0.8917

Epoch 00004: val_loss did not improve from 0.09708
Epoch 5/25
34/34 [==============================] - 36s 1s/step - loss: 0.2684 - accuracy: 0.9